## ECE495 Final Project: SNN and DNN on heart failure classification

Description here

#### Set up

**Ensure you are using your [495 Virtual Environment](https://github.com/kaitlin-fair/495venv_setup) before you begin!**  
    
Import Nengo and other supporting libraries into your program to get started:

In [1]:
import matplotlib.pyplot as plt
from urllib.request import urlretrieve
import tensorflow as tf
import nengo_dl
import nengo
from nengo.processes import Piecewise
from nengo.processes import WhiteSignal
from nengo.solvers import LstsqL2
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

### Datasets

Links and description

**load them below**


In [2]:
# Specify training data
file_name = "./heart.csv"

# Read the CSV file into a DataFrame
df = pd.read_csv(file_name)


# extract training data
y = df["output"]
X = df.drop(columns="output")

### Preprocessing
`Standardization` is a preprocessing technique that rescales the features so that they have the properties of a `standard normal distribution with a mean of 0 and a standard deviation of 1`. This is important in machine learning because it ensures that the features are on a similar scale, preventing some features from dominating the learning process simply because they have larger magnitudes.

The `StandardScaler` in scikit-learn works by calculating the mean and standard deviation of each feature in the training set and then transforming the data based on these statistics. The formula for standardization is:

$$ Standardized Value = \frac{Original Value − Mean}{ Standard Deviation}$$

The purpose of standardization is to make the features of the dataset comparable and to ensure that they all contribute equally to the model training. It is particularly important when working with algorithms that are sensitive to the scale of the input features, such as gradient-based optimization algorithms used in neural networks.

In [3]:
scaler = StandardScaler()
X = scaler.fit_transform(X)
# split
train_images, test_images, train_labels, test_labels = train_test_split(
    X, y, test_size=0.2, random_state=42
)

### SNN
Using SNN to classify datasets

### DNN
4 128 Neuron layers, Dropout of .03, 4 64 neuron layers, final layer with sigmoid activation


In [7]:
from keras import layers

model = tf.keras.models.Sequential(
    [
        layers.Flatten(),
        layers.Dense(128, activation="relu", name="Dense1"),
        layers.Dense(128, activation="relu", name="Dens2"),
        layers.Dense(128, activation="relu", name="Dense3"),
        layers.Dense(128, activation="relu", name="Dense4"),
        layers.Dropout(0.03),
        layers.Dense(64, activation="relu", name="Dense5"),
        layers.Dense(64, activation="relu", name="Dense6"),
        layers.Dense(64, activation="relu", name="Dense7"),
        layers.Dense(64, activation="relu", name="Dense8"),
        layers.Dense(2, activation="sigmoid", name="Final"),
    ]
)

# Compile
model.compile(
    optimizer="Adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

# Train
model.fit(train_images, train_labels, epochs=100)

# Eval
model.evaluate(test_images, test_labels, verbose=2)

# Predict
probability_model = tf.keras.Sequential([model, layers.Softmax()])
predictions = probability_model.predict(test_images)

# Analyze
predictions2 = np.argmax(predictions, axis=1)

rpt = classification_report(test_labels, predictions2)
print(rpt)

Epoch 1/100


c:\Users\C24Ryan.Lilly\Documents\ECE\448\envs\495venv\lib\site-packages\keras\backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


8/8 [==============================] - 1s 5ms/step - loss: 0.6790 - accuracy: 0.6405
Epoch 2/100
8/8 [==============================] - 0s 4ms/step - loss: 0.5309 - accuracy: 0.8347
Epoch 3/100
8/8 [==============================] - 0s 4ms/step - loss: 0.4018 - accuracy: 0.8347
Epoch 4/100
8/8 [==============================] - 0s 5ms/step - loss: 0.3699 - accuracy: 0.8430
Epoch 5/100
8/8 [==============================] - 0s 5ms/step - loss: 0.2936 - accuracy: 0.8967
Epoch 6/100
8/8 [==============================] - 0s 4ms/step - loss: 0.2573 - accuracy: 0.9215
Epoch 7/100
8/8 [==============================] - 0s 5ms/step - loss: 0.2298 - accuracy: 0.9215
Epoch 8/100
8/8 [==============================] - 0s 3ms/step - loss: 0.1857 - accuracy: 0.9504
Epoch 9/100
8/8 [==============================] - 0s 4ms/step - loss: 0.1632 - accuracy: 0.9545
Epoch 10/100
8/8 [==============================] - 0s 4ms/step - loss: 0.1382 - accuracy: 0.9587
Epoch 11/100
8/8 [=======================